# Random Forest Classifier With Pipeline and Hyperparameter tuning Implementation

In [1]:
import seaborn as sns
df=sns.load_dataset('tips')

In [2]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [4]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [6]:
df['day'].unique()

['Sun', 'Sat', 'Thur', 'Fri']
Categories (4, object): ['Thur', 'Fri', 'Sat', 'Sun']

In [7]:
df['time'].value_counts()

Dinner    176
Lunch      68
Name: time, dtype: int64

In [8]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [9]:
df.time

0      Dinner
1      Dinner
2      Dinner
3      Dinner
4      Dinner
        ...  
239    Dinner
240    Dinner
241    Dinner
242    Dinner
243    Dinner
Name: time, Length: 244, dtype: category
Categories (2, object): ['Lunch', 'Dinner']

In [10]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [11]:
df.time

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int64

In [12]:
df['time'].unique()

array([0, 1])

In [13]:
##  Divide the dataset into independent and dependent features
X=df.drop(labels=['time'],axis=1)
y=df.time

In [14]:
X['day'].value_counts()

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

In [15]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer ## to handle missing values
from sklearn.preprocessing import StandardScaler ## for feature scaling
from sklearn.preprocessing import OneHotEncoder ## categorical to numerical
from sklearn.compose import ColumnTransformer

In [18]:
categorical_cols = ['sex','smoker','day']
numerical_cols = ['total_bill','tip','size']

In [19]:
## Feature Engineering Automation
## Numerical pipeline
num_pipeline = Pipeline(
        steps=[
            ('imputer',SimpleImputer(strategy='median')), ## to handle missing values 
            ('scaler',StandardScaler()) ## to perform feature scaling
        ]
)

## categorical pipeline
cat_pipeline=Pipeline(
            steps=[
            ('imputer',SimpleImputer(strategy='most_frequent')), ## to handle missing values
            ('onehotencoder',OneHotEncoder()) ## to do feature Encoding --> categorical to numerical   
            ]
)

In [20]:
preprocessor =ColumnTransformer([
            ('num_pipeline',num_pipeline,numerical_cols),
            ('cat_pipeline',cat_pipeline,categorical_cols)
                                ])

In [21]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [22]:
from sklearn.ensemble import RandomForestClassifier

In [23]:
from sklearn.linear_model import LogisticRegression

In [24]:
from sklearn.tree import DecisionTreeClassifier

In [29]:
## Model Training automation
manymodel={
    'Random Forest':RandomForestClassifier(),
    'Logistic Regression': LogisticRegression(),
    'Decision Tree Classifier': DecisionTreeClassifier()
}

In [30]:
from sklearn.metrics import accuracy_score

In [31]:
def evaluate_model(X_train,y_train,X_test,y_test,manymodel):
    
    report={}
    for i in range(len(manymodel)):
        model=list(manymodel.values())[i]
        ## Train Model
        model.fit(X_train,y_train)
        
        ## Predict the testing Data
        y_test_pred = model.predict(X_test)
        
        ## get the accuracy score of the test data
        test_model_score = accuracy_score(y_test,y_test_pred)
        
        report[list(manymodel.keys())[i]] = test_model_score
        
        return report

In [32]:
evaluate_model(X_train,y_train,X_test,y_test,models)

{'Random Forest': 0.9672131147540983}

In [33]:
classifier=RandomForestClassifier()

In [37]:
## Hyper parameter tuning
params={
'max_depth':[3,5,10,None],
    'n_estimators':[100,200,300],
    'criterion':['gini','entropy']
}


In [38]:
from sklearn.model_selection import RandomizedSearchCV
cv= RandomizedSearchCV(classifier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)

In [39]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.946 total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.973 total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=None, n_estimators=100;, score=1.000 total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.917 total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=None, n_estimators=100;, score=0.917 total time=   0.2s
[CV 1/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.946 total time=   0.6s
[CV 2/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.973 total time=   0.6s
[CV 3/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.973 total time=   0.6s
[CV 4/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.917 total time=   0.6s
[CV 5/5] END criterion=entropy, max_depth=3, n_estimators=300;, score=0.944 total ti

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [40]:
cv.best_params_

{'n_estimators': 300, 'max_depth': 3, 'criterion': 'entropy'}